## Hyperdrive Exploration Simulation
This notebook is to explore the base functionality of Hyperdrive

For details on the simulation framework, please see our <a href="https://elfpy.element.fi/"> simulation documentation</a>

### Install repo requirements & import packages

In [14]:
# test: skip-cell
try: # install dependencies only if running on google colab
    import google.colab # check if running in colab
    !pip install -r https://raw.githubusercontent.com/element-fi/elf-simulations/main/requirements-3.8.txt 
    !pip install git+https://github.com/element-fi/elf-simulations.git
except:
    print("running locally & trusting that you have the dependencies installed")

running locally & trusting that you have the dependencies installed


In [15]:
from __future__ import annotations

import time as time

import numpy as np
from numpy.random._generator import Generator
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from elfpy import WEI
from elfpy.types import MarketActionType, MarketAction, Quantity, TokenType, MarketDeltas
from elfpy.simulators import Simulator
from elfpy.agent import Agent
from elfpy.markets import Market
from elfpy.utils import sim_utils
import elfpy.utils.time as time_utils
import elfpy.utils.outputs as output_utils
import elfpy.utils.post_processing as post_processing
from elfpy.types import MarketState, StretchedTime, Config
from elfpy.utils.outputs import get_gridspec_subplots

### Setup experiment parameters

In [16]:
config = Config()

# General config parameters
config.title = "Hyperdrive Core Features"
config.pricing_model_name = "Hyperdrive" # can be yieldspace or hyperdrive

config.num_trading_days = 20 # Number of simulated trading days
config.num_blocks_per_day = 5 # Blocks in a given day (7200 means ~12 sec per block)
config.num_position_days = 10 # How long a token reaches maturity

config.trade_fee_percent = 0.05 # fee percent collected on trades
config.redemption_fee_percent = 0.05 # fee collected on the spread of the flat portion

config.target_pool_apr = 0.01 # target pool APR of the initial market after the LP
config.target_liquidity = 500_000_000 # target total liquidity of the initial market, before any trades

config.log_level = output_utils.text_to_log_level("INFO") # Logging level, should be in ["DEBUG", "INFO", "WARNING"]
config.log_filename = "./hyperdrive_exploration.log" # Output filename for logging

config.shuffle_users = True

# Notebook specific parameters

# TBD

### Setup agents

In [17]:
class ShortAgent(Agent):
    """Opens Basic Short"""
    def action(self, market: Market):
        """implement user strategy"""
        return []

In [18]:
class LongAgent(Agent):
    """Opens Basic Long"""
    def action(self, market: Market):
        """implement user strategy"""
        return []

In [19]:
class LPAgent(Agent):
    """Adds a large LP"""
    def action(self, market: Market):
        """implement user strategy"""
        if self.wallet.lp_tokens > 0: # has already opened the lp
            action_list = []
        else:
            action_list = [
                self.create_agent_action(
                    action_type=MarketActionType.ADD_LIQUIDITY, trade_amount=self.budget
                ),
            ]
        return action_list

### Instantiate Example Agents

In [20]:

def get_example_agents(rng: Generator, num_shorts: int, num_longs: int, existing_agents: int = 0) -> list[Agent]:
    """Instantiate a set of custom agents"""
    agents = []
    for address in range(existing_agents, existing_agents + num_shorts):
        agent = ShortAgent(
            wallet_address=address,
            budget=5_000,
        )
        agent.log_status_report()
        agents += [agent]
    existing_agents += len(agents)
    return agents

### Setup simulation objects

In [21]:
# define root logging parameters
output_utils.setup_logging(log_filename=config.log_filename, log_level=config.log_level)

# get an instantiated simulator object
simulator = sim_utils.get_simulator(config)

### Run the simulation

In [22]:
# add the random agents
trading_agents = get_example_agents(
    rng=simulator.rng,
    num_shorts=1,
    num_longs=0,
    existing_agents=len(simulator.agents)
)
simulator.add_agents(trading_agents)
print(f"Simulator has {len(simulator.agents)} agents")

# run the simulation
simulator.run_simulation()

Simulator has 2 agents


IndexError: list index out of range